## Import

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Read csv

In [ ]:
file = "Pure/pure_pids.csv"
df = pd.read_csv(file, usecols=['objid', 'bands', 'pid1', 'pid2'], dtype=object)

## Band-Count and Object-Bands Dicts

In [ ]:
u, g, r, i = 0, 0, 0, 0
u_ids, g_ids, r_ids, i_ids = [], [], [], []

band_stat = {b:[0, []] for b in "ugri"}
objid_stat = {}

for i in range(len(df)) :
    objid = df.loc[i, 'objid']
    bands = df.loc[i, 'bands']
    for b in bands :
        band_stat[b][0] += 1
        band_stat[b][1].append(df.loc[i, 'objid'])
        
    objid_stat[objid] = bands

## Band Correlation

In [ ]:
corrs = {}
for b1 in "ugri" :
    corr = {b:0 for b in "ugri" if b != b1}

    other_count = []
    single_band = 0
    for objid in band_stat[b1][1] :
        other = 0
        for b2 in objid_stat[objid] :
            if b1 != b2 :
                other += 1
                corr[b2] += 1

        if not other :
            single_band += 1
        else :
            other_count.append(other)

    other_count = np.unique(np.array(other_count), return_counts=True)
    corrs[b1] = (corr, single_band, other_count)